In [1]:
"""
This section imports the necessary libraries.
qiskit is the main library for quantum computing,
AerSimulator is used for simulating quantum circuits,
and numpy is used for numerical operations like generating random numbers.
"""

from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
import qiskit_aer.noise as noise
from qiskit_aer.noise import NoiseModel
from qiskit_ibm_provider import IBMProvider

from math import ceil, floor
import pandas as pd
import time

/opt/conda/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/opt/conda/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
provider = IBMProvider()

# Get the backend (quantum device)
backend_qc = provider.get_backend('ibm_kyoto')

In [3]:
# Get the noise model from the backend
noise_model = NoiseModel.from_backend(backend_qc)

# Get coupling map from the backend
coupling_map = backend_qc.configuration().coupling_map

prob_h = 0.002  # Hypothetical error rate for H gate, slightly higher than typical single-qubit gate errors
prob_cx = 0.015  # Hypothetical error rate for CZ, estimated from CX gate error rates

error_h = noise.depolarizing_error(prob_h, 1) # Creating a 1 qubit error for the H gate
error_cz = noise.depolarizing_error(prob_cx, 2) # Creating a 2 qubit error for the CZ and CX gates

# Add these to the noise model
noise_model.add_all_qubit_quantum_error(error_h, ['h'])
noise_model.add_all_qubit_quantum_error(error_cz, ['cx'])

# Fetching the other gates from the backend
basis_gates = noise_model.basis_gates

# Perform a noise simulation
backend = AerSimulator(noise_model=noise_model,
                       coupling_map=coupling_map,
                       basis_gates=basis_gates)

In [4]:
class QuantumEntanglement:
    """
    This class achieves quantum teleportation of either the |0> state or the
    |1> state. It depends on the number of qubits used for the teleportation;
    if 2 qubits are used then the teleportation is done using Bell states, if
    more than 2 qubits are used then the teleportation is done using GHZ states.
    """
    def __init__(self, qubits=2, beta_state='00', backend=backend):
        """
        This function initializes the entire class.
        
        Parameters:
        --------------
        
        qubits :              The number of qubits used for the teleportation
        
        initial_state :       The state that will be teleported
        
        teleportation_state : The entangled state that will be used to achieve
                              the teleportation
        """
        # Making the number of qubits a property of self
        self.M = qubits
        
        # Creating a quantum circuit with M+1 qubits and M+1 classical bits
        self.circuit = QuantumCircuit(self.M, self.M)
        
        # Establishing the state we will use for teleportation
        self.teleportation_state = beta_state  # (either '00', '01', '10', or '11')
        
        # Set the backend
        self.backend = backend
        
    def create_entangled_state(self):
        """
        This function creates the entanglement between the last M-1 qubits.
        """
        # We first consider which teleportation state we will be using, these are
        # the 'beta' states
        if self.teleportation_state[0] == '1':
            self.circuit.x(0) # If x = 1, we change qubit 1 to the |1> state
            
        if self.teleportation_state[1] == '1':
            for i in range(floor(self.M/2)):
                self.circuit.x(i+ceil(self.M/2)) # If y = 1, we change the last floor(M/2) qubits to the |1> state
        
        # Performing the steps necessary for entanglement
        self.circuit.h(0) # First apply the Hadamard gate to qubit 0
        for qubit in range(self.M-1):
            self.circuit.cx(0, qubit+1) # Then apply a C-Not gate between qubit 0 and all the remaining qubits
        
        # Measuring all qubits into classical bits (except for qubit M+1)
        qubit_list = list(range(0, self.M))
        self.circuit.measure(qubit_list, qubit_list)

    def execute_circuit(self):
        """
        This function executes the entire circuit built by the previous functions,
        then runs it on the backend.
        """
        if self.backend is None:
            raise ValueError("Backend must be set before executing the circuit.")
        
        # Calling all functions from above
        self.create_entangled_state()
        
        # Running the circuit on the backend
        result = self.backend.run(self.circuit, shots=10000).result()
        
        # Getting the measurement outcomes from the circuit
        counts = result.get_counts(self.circuit)
        
        return counts # Return the counts dictionary

In [5]:
def check_success(counts):
    measurements = list(counts.values())
    if len(measurements) == 2 and abs(measurements[0] - measurements[1]) < 200:
        return True
    return False

In [6]:
# M qubits in the teleportation, M+1 qubits total
M = 15

# Listing all different entanglement states
beta_states = ['00', '10', '01', '11']
qubits = list(range(2, M))

data = []

current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
print(f"Starting: {current_time}")

for qubit in qubits:
    for beta in beta_states:
        # Create a QuantumEntanglement object for each combination
        entanglement = QuantumEntanglement(qubits=qubit, beta_state=beta)
        
        # Execute the circuit and get counts
        counts = entanglement.execute_circuit()
        
        # Check for success of entanglement
        success = check_success(counts)
        
        for measurement, frequency in counts.items():
            data.append({
                'Qubits': qubit,
                'Beta State': beta,
                'Success': success,
                'Measurement': measurement,
                'Frequency': frequency
            })

    current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
    print(f"Update: Completed {qubit} qubits at {current_time}")
        
current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
print(f"Finished job: {current_time}")

Starting: 2024-04-28 01:55:46
Update: Completed 2 qubits at 2024-04-28 01:56:03
Update: Completed 3 qubits at 2024-04-28 01:56:18
Update: Completed 4 qubits at 2024-04-28 01:56:33
Update: Completed 5 qubits at 2024-04-28 01:56:48
Update: Completed 6 qubits at 2024-04-28 01:57:03
Update: Completed 7 qubits at 2024-04-28 01:57:19
Update: Completed 8 qubits at 2024-04-28 01:57:36
Update: Completed 9 qubits at 2024-04-28 01:57:54
Update: Completed 10 qubits at 2024-04-28 01:58:13
Update: Completed 11 qubits at 2024-04-28 01:58:41
Update: Completed 12 qubits at 2024-04-28 01:59:44
Update: Completed 13 qubits at 2024-04-28 02:02:57
Update: Completed 14 qubits at 2024-04-28 02:11:37
Finished job: 2024-04-28 02:11:37


In [7]:
# Converting the dictionary to a pandas DataFrame for better visualization and analysis
df = pd.DataFrame(data)

# To save the DataFrame to a CSV file
df.to_csv('entanglement_results_noise.txt', index=False)